# Watch Me Code 3: Mapping with Folium

Folium is a Python wrapper library for the OpenStreetMaps api. It allows you to place data on a map in a variety of ways.


In [1]:
! pip install folium

In [1]:
import folium
import pandas as pd
import random 
from IPython.display import display
from ipywidgets import interact_manual

In [2]:
# we need to center the map in the middle of the US. I googled for the location.
CENTER_US = (39.8333333,-98.585522)
london_location = (51.5074, -0.1278)
@interact_manual(london=False)
def onclick(london):
    if london:
        map = folium.Map(location=london_location, zoom_start=20)
    else:
        map = folium.Map(location=CENTER_US, zoom_start=4)
    display(map)
    

interactive(children=(Checkbox(value=False, description='london'), Button(description='Run Interact', style=Bu…

In [5]:
france = (47.7516, 1.6751)
france_map = folium.Map(location=france,zoom_start=6)
france_map

## Map Pins

In [6]:
# read in a data file of IP address to locations.
data = pd.read_csv('https://raw.githubusercontent.com/mafudge/datasets/master/clickstream/ip_lookup.csv')
data.sample(5)

,IP,Country,State,City,ApproxLat,ApproxLng
7,74.111.6.173,USA,VA,Arlington,38.879970,-77.106770
8,128.230.122.180,USA,NY,Syracuse,43.048122,-76.147424
22,8.37.71.57,USA,CA,Los Angeles,34.052234,-118.243685
20,8.37.71.69,USA,CA,Los Angeles,34.052234,-118.243685
14,8.37.70.77,USA,CA,Los Angeles,34.052234,-118.243685


In [7]:
from IPython.display import display
us = folium.Map(location=CENTER_US, zoom_start=4)

# Let's place each location on the map
for row in data.to_records():
    pos = (row['ApproxLat'],row['ApproxLng'])
    marker = folium.Marker(location=pos, 
                    popup=f"{row['City']},<b>{row['State']}</b>"
                          )
    us.add_child(marker)
display(us)

In [20]:
# Same thing with a different icon and colors. Icons come from http://fontawesome.io/icons/ but its an older version.
map = folium.Map(location=CENTER_US, zoom_start=4)

colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
            'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
            'darkpurple', 'pink', 'lightblue', 'lightgreen',
            'gray', 'black', 'lightgray']
for row in data.to_records():
    pos = (row['ApproxLat'],row['ApproxLng'])
    marker = folium.Marker(location=pos, 
                    popup="%s, %s" % (row['City'],row['State']),
                    icon = folium.Icon(color = random.choice(colors), icon='arrow-up')
                          )
    map.add_child(marker)
map

In [9]:
# There are other map tiles available. See https://folium.readthedocs.io/en/latest/quickstart.html
# Instead of Markers we use circles colors are HTML color codes http://htmlcolorcodes.com/
CENTER_US = (39.8333333,-98.585522)
map2 = folium.Map(location=CENTER_US, zoom_start=4)
for row in data.to_records():
    map2.add_child(folium.CircleMarker(location=(row['ApproxLat'],row['ApproxLng']), 
                    popup=row['City'], radius=10, color='#0000FF',  fill_color='#FF3333'))
    
map2

# Choropleths

Choropleths are cartographic overlays based on boundries defined in a geo JSON file.


In [21]:
# State level geo-json overlay choropleth
CENTER_US = (39.8333333,-98.585522)
state_geojson = 'WMC3-us-states.json'
map3 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Open Street Map')
map3.choropleth(geo_data=state_geojson)
map3


/opt/conda/lib/python3.9/site-packages/folium/folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [22]:
states = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')
state_counts = pd.DataFrame( {'Counts' : data['State']. value_counts() } ).sort_index()
state_counts['StateCode'] = state_counts.index
state_data = states.merge(state_counts, how="left", left_on='Abbreviation', right_on='StateCode')
state_data = state_data[['Abbreviation','Counts']]
state_data = state_data.fillna(0)
state_data


,Abbreviation,Counts
0,AL,0.0
1,AK,0.0
2,AZ,0.0
3,AR,0.0
4,CA,10.0
5,CO,0.0
6,CT,0.0
7,DE,0.0
8,DC,0.0
9,FL,1.0


In [23]:
CENTER_US = (39.8333333,-98.585522)
state_geojson = 'WMC3-us-states.json'
map3 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Open Street Map')
folium.Choropleth(geo_data=state_geojson,data=state_data, columns=['Abbreviation','Counts'], 
                key_on ='feature.id', fill_color='BuGn', legend_name='Website Visitors').add_to(map3)
map3

In [24]:
# Here's a more straigtforward example with unemployment data:
unemployment = pd.read_csv('https://raw.githubusercontent.com/wrobstory/vincent/master/examples/data/US_Unemployment_Oct2012.csv')
state_geojson = 'WMC3-us-states.json'
map4 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Open Street Map')
folium.Choropleth(geo_data=state_geojson,data=unemployment, 
                columns=['State','Unemployment'], key_on ='feature.id', fill_color='YlGn', 
                legend_name='2012 US Unemployment Rate %').add_to(map4)
map4
